# Machine Learning II Project

Group Members:
- Daniel Kruk 20211687
- Marcelo Junior 20211677

# Libraries

In [61]:
#Package Importation
import pandas as pd
import numpy as np
import re
from datetime import datetime


# Import Datasets

In [34]:
# Dataset Importation
path = '/Users/marcelojunior/Documents/Nova IMS - Lisboa (2021 - 2024)/4º Semestre/Machine Learning II.nosync/KeizerKrukis/Project Data/'

# Customer Info CSV
customer_info = pd.read_csv(path + "Customer Info Dataset.csv")

# Customer Basket CSV
customer_basket = pd.read_csv(path + "Customer Basket Dataset.csv")

# Product Mapping Excel File
product_mapping = pd.read_excel(path + "Product Mapping Excel File.xlsx")

We can now see the info and data types of the datasets.

In [57]:
display(customer_info.info())
print('----' * 20 + '\n')
display(customer_basket.info())
print('----' * 20 + '\n')
display(product_mapping.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   customer_id                              30000 non-null  int64  
 1   customer_name                            30000 non-null  object 
 2   highest_education                        30000 non-null  object 
 3   customer_gender                          30000 non-null  object 
 4   customer_birthdate                       30000 non-null  object 
 5   kids_home                                30000 non-null  float64
 6   teens_home                               30000 non-null  float64
 7   number_complaints                        30000 non-null  float64
 8   distinct_stores_visited                  30000 non-null  float64
 9   lifetime_spend_groceries                 30000 non-null  float64
 10  lifetime_spend_electronics               30000

None

--------------------------------------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    80000 non-null  int64 
 1   invoice_id     80000 non-null  int64 
 2   list_of_goods  80000 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


None

--------------------------------------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  164 non-null    object
 1   category      164 non-null    object
dtypes: object(2)
memory usage: 2.7+ KB


None

After importing the datasets, we can take a look at them and look out for anything out of the ordinary that may be present, alongside taking note of their organisation.

In [35]:
customer_info

,customer_id,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
0,4239,Sara Moscowitz,female,09/10/1999 03:11 PM,0.0,0.0,0.0,2.0,91.0,20.0,...,53.0,48.0,47.0,105.0,85.0,0.192154,2017.0,NaN,38.754263,-9.163397
1,19330,Donald Robertson,male,06/26/1981 10:58 PM,1.0,0.0,0.0,2.0,197.0,5958.0,...,86.0,80.0,45.0,1964.0,85.0,0.098647,2015.0,NaN,38.777761,-9.156197
2,2272,Phd. Felicia Bradshaw,female,01/31/1987 09:08 AM,1.0,1.0,0.0,20.0,397.0,20.0,...,204.0,170.0,46.0,5.0,408.0,0.497251,2008.0,NaN,38.785329,-9.127113
3,25288,Emma Williams,female,06/17/1942 12:02 AM,3.0,4.0,1.0,2.0,5282.0,237.0,...,1081.0,1068.0,617.0,1078.0,1956.0,0.206713,2010.0,NaN,38.744086,-9.156589
4,12829,Hilda Huerta,female,01/31/2000 11:43 AM,0.0,0.0,0.0,2.0,82.0,22.0,...,39.0,39.0,48.0,103.0,83.0,0.200625,2015.0,NaN,38.756024,-9.162970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,625,Phd. Karen Watts,female,11/01/1974 05:26 AM,0.0,0.0,1.0,2.0,209.0,5474.0,...,21.0,11.0,57.0,1984.0,112.0,0.096366,2008.0,935162.0,38.734446,-9.171911
29996,26447,Curtis Thompson,male,02/21/1986 11:27 AM,0.0,0.0,0.0,3.0,185.0,1066.0,...,47.0,41.0,50.0,197.0,22.0,0.118998,2014.0,NaN,38.767070,-9.144301
29997,182,Richard Carter,male,01/14/2001 05:49 AM,0.0,0.0,0.0,2.0,83.0,23.0,...,55.0,52.0,51.0,94.0,77.0,0.199432,2016.0,953891.0,38.728487,-9.164140
29998,13768,Jerry Yoder,male,08/21/1958 12:12 PM,1.0,0.0,1.0,20.0,391.0,20.0,...,160.0,132.0,54.0,5.0,398.0,0.494527,2006.0,NaN,38.751123,-9.182512


# Basic Preprocessing <br>
**Customer Info dataset** <br>
Customer education <br>
 <br>


**Customer Info dataset** <br>
replace customer card info / add a different column -> customer card as binary (yes or no/ 1/0) <br>

### Customer Info - Education
It could be noticed that some customers start their names with their highest education level (Bsc., Msc. and Phd.). As this does not provide any value in the `customer_name` field, we can create a new column called `highest_education` which will provide us that information in a meaningful manner. <br>
We will first see the first 4 letters of every name, to see how are the education levels abbreviated (mostly to know about the Master degree, as MBA and MSc are some of the options it could take).

In [39]:
prefs = customer_info['customer_name'].str.slice(stop=4)

# Print the value counts of the first 4 letters
prefs.value_counts()

Bsc.    3815
Phd.    3810
Msc.    3807
Mich     354
Robe     341
        ... 
Babe       1
Juni       1
Ashe       1
Vito       1
Jae        1
Name: customer_name, Length: 1445, dtype: int64

As can be seen, the educaiton levels are `Bsc.`, `Msc.` and `Phd.`. <br>
We will now procced to create the new column, assigning a Null value for those which did not include it in their name.

In [48]:
# Create a copy for safety
ci_copy = customer_info.copy()

# List of educational levels previously seen to be extracted
edu_levels = ['Bsc.', 'Msc.', 'Phd.']

# Create a regular expression pattern to match the educational level
pattern = '|'.join(edu_levels)

# Extract the educational level
ci_copy['highest_education'] = ci_copy['customer_name'].str.extract('(' + pattern + ')')

# Remove the educational level from the 'customer_name' column
ci_copy['customer_name'] = ci_copy['customer_name'].str.replace(pattern, '').str.strip()

# Insert the 'education' column as the third column in the DataFrame
ci_copy.insert(2, 'highest_education', ci_copy.pop('highest_education'))

# To avoid having many NaNs, it was decided to give them the designation 'Missing'
ci_copy['highest_education'].fillna('Missing', inplace= True)

# See updated Customer Info
ci_copy

/var/folders/wv/8j5v8n4s4lq6l7yj4cl1lrkm0000gp/T/ipykernel_77356/708447921.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  ci_copy['customer_name'] = ci_copy['customer_name'].str.replace(pattern, '').str.strip()


,customer_id,customer_name,highest_education,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
0,4239,Sara Moscowitz,Missing,female,09/10/1999 03:11 PM,0.0,0.0,0.0,2.0,91.0,...,53.0,48.0,47.0,105.0,85.0,0.192154,2017.0,NaN,38.754263,-9.163397
1,19330,Donald Robertson,Missing,male,06/26/1981 10:58 PM,1.0,0.0,0.0,2.0,197.0,...,86.0,80.0,45.0,1964.0,85.0,0.098647,2015.0,NaN,38.777761,-9.156197
2,2272,Felicia Bradshaw,Phd.,female,01/31/1987 09:08 AM,1.0,1.0,0.0,20.0,397.0,...,204.0,170.0,46.0,5.0,408.0,0.497251,2008.0,NaN,38.785329,-9.127113
3,25288,Emma Williams,Missing,female,06/17/1942 12:02 AM,3.0,4.0,1.0,2.0,5282.0,...,1081.0,1068.0,617.0,1078.0,1956.0,0.206713,2010.0,NaN,38.744086,-9.156589
4,12829,Hilda Huerta,Missing,female,01/31/2000 11:43 AM,0.0,0.0,0.0,2.0,82.0,...,39.0,39.0,48.0,103.0,83.0,0.200625,2015.0,NaN,38.756024,-9.162970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,625,Karen Watts,Phd.,female,11/01/1974 05:26 AM,0.0,0.0,1.0,2.0,209.0,...,21.0,11.0,57.0,1984.0,112.0,0.096366,2008.0,935162.0,38.734446,-9.171911
29996,26447,Curtis Thompson,Missing,male,02/21/1986 11:27 AM,0.0,0.0,0.0,3.0,185.0,...,47.0,41.0,50.0,197.0,22.0,0.118998,2014.0,NaN,38.767070,-9.144301
29997,182,Richard Carter,Missing,male,01/14/2001 05:49 AM,0.0,0.0,0.0,2.0,83.0,...,55.0,52.0,51.0,94.0,77.0,0.199432,2016.0,953891.0,38.728487,-9.164140
29998,13768,Jerry Yoder,Missing,male,08/21/1958 12:12 PM,1.0,0.0,1.0,20.0,391.0,...,160.0,132.0,54.0,5.0,398.0,0.494527,2006.0,NaN,38.751123,-9.182512


In [50]:
ci_copy['highest_education'].value_counts()

Missing    18568
Bsc.        3815
Phd.        3810
Msc.        3807
Name: highest_education, dtype: int64

In [54]:
# Apply the changes to original customer_info
customer_info = ci_copy.copy()

customer_info

,customer_id,customer_name,highest_education,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
0,4239,Sara Moscowitz,Missing,female,09/10/1999 03:11 PM,0.0,0.0,0.0,2.0,91.0,...,53.0,48.0,47.0,105.0,85.0,0.192154,2017.0,NaN,38.754263,-9.163397
1,19330,Donald Robertson,Missing,male,06/26/1981 10:58 PM,1.0,0.0,0.0,2.0,197.0,...,86.0,80.0,45.0,1964.0,85.0,0.098647,2015.0,NaN,38.777761,-9.156197
2,2272,Felicia Bradshaw,Phd.,female,01/31/1987 09:08 AM,1.0,1.0,0.0,20.0,397.0,...,204.0,170.0,46.0,5.0,408.0,0.497251,2008.0,NaN,38.785329,-9.127113
3,25288,Emma Williams,Missing,female,06/17/1942 12:02 AM,3.0,4.0,1.0,2.0,5282.0,...,1081.0,1068.0,617.0,1078.0,1956.0,0.206713,2010.0,NaN,38.744086,-9.156589
4,12829,Hilda Huerta,Missing,female,01/31/2000 11:43 AM,0.0,0.0,0.0,2.0,82.0,...,39.0,39.0,48.0,103.0,83.0,0.200625,2015.0,NaN,38.756024,-9.162970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,625,Karen Watts,Phd.,female,11/01/1974 05:26 AM,0.0,0.0,1.0,2.0,209.0,...,21.0,11.0,57.0,1984.0,112.0,0.096366,2008.0,935162.0,38.734446,-9.171911
29996,26447,Curtis Thompson,Missing,male,02/21/1986 11:27 AM,0.0,0.0,0.0,3.0,185.0,...,47.0,41.0,50.0,197.0,22.0,0.118998,2014.0,NaN,38.767070,-9.144301
29997,182,Richard Carter,Missing,male,01/14/2001 05:49 AM,0.0,0.0,0.0,2.0,83.0,...,55.0,52.0,51.0,94.0,77.0,0.199432,2016.0,953891.0,38.728487,-9.164140
29998,13768,Jerry Yoder,Missing,male,08/21/1958 12:12 PM,1.0,0.0,1.0,20.0,391.0,...,160.0,132.0,54.0,5.0,398.0,0.494527,2006.0,NaN,38.751123,-9.182512


### Birthdates
Instead of working with birthdates, it is better to work with `Age`, as it is a better measure to group customers. <br>
As it is stored as an `object`, a transformation into `timedate54` is needed to thus turn it into age based on a specified date.

In [63]:
# Create a copy for safety again
ci_copy = customer_info.copy()

# Define the date format and the specified date with the same format
date_format = "%m/%d/%Y %I:%M %p"

spec_date_str = '03/08/2023 12:00 PM'
spec_date = datetime.strptime(spec_date_str, date_format)


# Define a lambda function to calculate the age from their birth date
age_calc = lambda x: spec_date.year - datetime.strptime(x, date_format).year - ((spec_date.month, spec_date.day) < (datetime.strptime(x, date_format).month, datetime.strptime(x, date_format).day))

# Modify the customer_birthdate column and renaming it to Age
ci_copy['customer_birthdate'] = ci_copy['customer_birthdate'].apply(age_calc)
ci_copy = ci_copy.rename(columns={'customer_birthdate': 'Age'})

# See the updated DataFrame
ci_copy

,customer_id,customer_name,highest_education,customer_gender,Age,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
0,4239,Sara Moscowitz,Missing,female,23,0.0,0.0,0.0,2.0,91.0,...,53.0,48.0,47.0,105.0,85.0,0.192154,2017.0,NaN,38.754263,-9.163397
1,19330,Donald Robertson,Missing,male,41,1.0,0.0,0.0,2.0,197.0,...,86.0,80.0,45.0,1964.0,85.0,0.098647,2015.0,NaN,38.777761,-9.156197
2,2272,Felicia Bradshaw,Phd.,female,36,1.0,1.0,0.0,20.0,397.0,...,204.0,170.0,46.0,5.0,408.0,0.497251,2008.0,NaN,38.785329,-9.127113
3,25288,Emma Williams,Missing,female,80,3.0,4.0,1.0,2.0,5282.0,...,1081.0,1068.0,617.0,1078.0,1956.0,0.206713,2010.0,NaN,38.744086,-9.156589
4,12829,Hilda Huerta,Missing,female,23,0.0,0.0,0.0,2.0,82.0,...,39.0,39.0,48.0,103.0,83.0,0.200625,2015.0,NaN,38.756024,-9.162970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,625,Karen Watts,Phd.,female,48,0.0,0.0,1.0,2.0,209.0,...,21.0,11.0,57.0,1984.0,112.0,0.096366,2008.0,935162.0,38.734446,-9.171911
29996,26447,Curtis Thompson,Missing,male,37,0.0,0.0,0.0,3.0,185.0,...,47.0,41.0,50.0,197.0,22.0,0.118998,2014.0,NaN,38.767070,-9.144301
29997,182,Richard Carter,Missing,male,22,0.0,0.0,0.0,2.0,83.0,...,55.0,52.0,51.0,94.0,77.0,0.199432,2016.0,953891.0,38.728487,-9.164140
29998,13768,Jerry Yoder,Missing,male,64,1.0,0.0,1.0,20.0,391.0,...,160.0,132.0,54.0,5.0,398.0,0.494527,2006.0,NaN,38.751123,-9.182512


As everything looks fine, we can move this changes to the original dataframe.

In [64]:
customer_info = ci_copy.copy()
customer_info

,customer_id,customer_name,highest_education,customer_gender,Age,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
0,4239,Sara Moscowitz,Missing,female,23,0.0,0.0,0.0,2.0,91.0,...,53.0,48.0,47.0,105.0,85.0,0.192154,2017.0,NaN,38.754263,-9.163397
1,19330,Donald Robertson,Missing,male,41,1.0,0.0,0.0,2.0,197.0,...,86.0,80.0,45.0,1964.0,85.0,0.098647,2015.0,NaN,38.777761,-9.156197
2,2272,Felicia Bradshaw,Phd.,female,36,1.0,1.0,0.0,20.0,397.0,...,204.0,170.0,46.0,5.0,408.0,0.497251,2008.0,NaN,38.785329,-9.127113
3,25288,Emma Williams,Missing,female,80,3.0,4.0,1.0,2.0,5282.0,...,1081.0,1068.0,617.0,1078.0,1956.0,0.206713,2010.0,NaN,38.744086,-9.156589
4,12829,Hilda Huerta,Missing,female,23,0.0,0.0,0.0,2.0,82.0,...,39.0,39.0,48.0,103.0,83.0,0.200625,2015.0,NaN,38.756024,-9.162970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,625,Karen Watts,Phd.,female,48,0.0,0.0,1.0,2.0,209.0,...,21.0,11.0,57.0,1984.0,112.0,0.096366,2008.0,935162.0,38.734446,-9.171911
29996,26447,Curtis Thompson,Missing,male,37,0.0,0.0,0.0,3.0,185.0,...,47.0,41.0,50.0,197.0,22.0,0.118998,2014.0,NaN,38.767070,-9.144301
29997,182,Richard Carter,Missing,male,22,0.0,0.0,0.0,2.0,83.0,...,55.0,52.0,51.0,94.0,77.0,0.199432,2016.0,953891.0,38.728487,-9.164140
29998,13768,Jerry Yoder,Missing,male,64,1.0,0.0,1.0,20.0,391.0,...,160.0,132.0,54.0,5.0,398.0,0.494527,2006.0,NaN,38.751123,-9.182512


### Location
Another meaningful insight would be to find duplicate customers and distinct customers who possible share the same address. <br>
As such, we can join `latitude` and `longitude` and group the data by them.

In [80]:
# Group the data by longitude and latitude and count the number of occurrences of each unique combination
duplicate_coords = customer_info.groupby(['latitude', 'longitude']).filter(lambda x: len(x) > 1)

# print the duplicate coordinates
#print(duplicate_coords)
duplicate_coords

,customer_id,customer_name,highest_education,customer_gender,Age,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude


In [85]:
# create a boolean mask indicating which rows have duplicate coordinates
duplicates = customer_info.duplicated(['longitude'])

# filter the dataframe to show only the rows with duplicate coordinates
duplicate_rows = customer_info[duplicates]

duplicate_rows

,customer_id,customer_name,highest_education,customer_gender,Age,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,...,lifetime_spend_meat,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude
72,551,Jean Pinette,Bsc.,female,35,1.0,2.0,1.0,3.0,5065.0,...,1186.0,1045.0,497.0,1159.0,2143.0,0.196566,2007.0,961207.0,38.748029,-9.098630
98,16555,Frank White,Missing,male,45,2.0,3.0,1.0,2.0,5219.0,...,1213.0,1097.0,550.0,987.0,2024.0,0.206686,2013.0,NaN,38.763675,-9.098630
125,24734,Lee Chaffin,Missing,male,29,1.0,0.0,0.0,3.0,203.0,...,86.0,11.0,46.0,2100.0,117.0,0.100475,2013.0,NaN,38.708110,-9.152628
145,3321,Marsha Willard,Phd.,female,41,2.0,1.0,0.0,2.0,958.0,...,0.0,0.0,103.0,46.0,214.0,0.050491,2011.0,NaN,38.719242,-9.152628
170,11560,Maurine Purgason,Bsc.,female,71,1.0,0.0,1.0,2.0,189.0,...,103.0,110.0,65.0,2082.0,101.0,0.100011,2015.0,NaN,38.718880,-9.152628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29688,21252,Adam Morris,Missing,male,41,4.0,1.0,1.0,3.0,5133.0,...,1169.0,1203.0,484.0,1018.0,2185.0,0.202992,2009.0,NaN,38.745320,-9.098630
29739,1666,Marissa Hewitt,Bsc.,female,30,1.0,1.0,0.0,3.0,1044.0,...,0.0,0.0,107.0,51.0,205.0,0.049974,2009.0,NaN,38.712255,-9.152628
29792,11686,Herman Revilla,Phd.,male,80,5.0,3.0,1.0,3.0,4967.0,...,898.0,916.0,562.0,1176.0,2014.0,0.191917,2015.0,NaN,38.764272,-9.098630
29807,26548,Oliver Holliman,Missing,male,68,2.0,1.0,0.0,2.0,910.0,...,0.0,0.0,98.0,44.0,186.0,0.050411,2012.0,NaN,38.719799,-9.152628
